# A Tour of Digital Audio in Python/Numpy
### <a href = "https://www.ctralie.com">Chris Tralie</a> 

## Bio

### Upper Dublin Computer Club President 2005-2007

<img src = "Pics/acsl2007.jpg" width="50%">

### Graduated Upper Dublin 2007

<img src = "Pics/udgrad.jpg" width="50%">

### Princeton Class of 2011, Major in Electrical Engineering + Computer Science

<img src = "Pics/princeton.jpg" width="20%">

### Duke University Ph.D. in Electrical And Computer Engineering, 2017

<img src = "Pics/duke.jpg" width="50%">

### Assistant Professor in Math/CS At Ursinus College Since Fall 2019

<img src = "Pics/seraiah3.jpg" width="50%">

# Digital Audio: The Basics

I'll be giving some highlights of a course I'm currently teaching to Ursinus undergraduates.  Course web site at <a href = "https://ursinus-cs372-s2023.github.io/CoursePage/">this link</a>


In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import IPython.display as ipd

First, what is audio?  Let's load some in and find out!

In [ ]:
sr, x = wavfile.read("Audio/femalecountdown.wav")
x = x/32768
plt.figure()
plt.plot(x)
ipd.Audio(x, rate=sr)

In [ ]:
# Show sample rate

In [ ]:
# Show slicing

In [ ]:
# Show reversing

<a href = "https://ursinus-cs372-s2023.github.io/CoursePage/ClassExercises/Week1/Week1_AudioReverseGame/">Audio reversing game</a>

## Filtering Audio

In [ ]:
# Windowed energy
def get_energy(x, win):
    """
    Compute the energy in a window around each sample
    
    Parameters
    ----------
    x: ndarray(N)
        Audio samples
    win: int
        Window length in which to sum energy
    
    Returns
    -------
    ndarray(N): Energy arond each sample
    """
    eng = x**2
    w = np.zeros(win//2)
    eng = np.concatenate((w, eng, w))
    eng = np.cumsum(eng)
    return eng[win::] - eng[0:-win]


sr, x = wavfile.read("Audio/femalecountdown.wav")
x = x/32768
eng = get_energy(x, 1000)

plt.figure()
plt.subplot(211)
plt.plot(x)
plt.ylabel("Audio Waveform")
plt.subplot(212)
plt.plot(eng)
plt.xlabel("Sample Index")
plt.ylabel("Energy")
plt.tight_layout()

In [ ]:
# Filter by energy

In [ ]:
# Windowed zero crossings
def get_zcs(x, win):
    """
    Compute the zero crossings in a window around each sample
    
    Parameters
    ----------
    x: ndarray(N)
        Audio samples
    win: int
        Window length in which to sum energy
    
    Returns
    -------
    ndarray(N): Zero crossing count arond each sample
    """
    zcs = np.sign(x[1::]) - np.sign(x[0:-1]) == 2
    w = np.zeros(win//2)
    zcs = np.concatenate((w, [0], zcs, w))
    zcs = np.cumsum(zcs)
    return zcs[win::] - zcs[0:-win]

sr, x = wavfile.read("Audio/femalecountdown.wav")
x = x/32768
zcs = get_zcs(x, 4000)

plt.figure()
plt.subplot(211)
plt.plot(x)
plt.ylabel("Audio Waveform")
plt.subplot(212)
plt.plot(eng)
plt.xlabel("Sample Index")
plt.ylabel("Zero Crossings")
plt.tight_layout()

In [ ]:
# Filter by zero crossings


## Periodic Sounds/Pitches

Define period, define frequency, and show cosine

In [ ]:
# Show cosine sampled at a particular period

# What do we notice when the frequency goes up?

Equation for pitch perception

In [1]:
# Make happy birthday tune with cosines

In [2]:
# Make happy birthday tune with square waves

In [3]:
# Make happy birthday tune with triangle waves

# Convolution / Echoes

<img src = "Convolution.gif">